In [ ]:
import numpy as np
from numpy import linalg as LA
import cv2
import matplotlib.pyplot as plt
import random
%matplotlib inline

# Question 1)

In [ ]:
img1 = cv2.imread('GB1.jpg', 0) 
img2 = cv2.imread('GB2.jpg', 0)

img1_ori = cv2.imread('GB1.jpg') 
img2_ori = cv2.imread('GB2.jpg')

img1_ori = cv2.cvtColor(img1_ori, cv2.COLOR_BGR2RGB)
img2_ori = cv2.cvtColor(img2_ori, cv2.COLOR_BGR2RGB)

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# BFMatcher with default params
bf = cv2.BFMatcher( crossCheck=True)
matches = bf.match(des1,des2)

print "img1 has {} keypoints".format(len(des1))
print "img2 has {} keypoints".format(len(des2))
print "two images has {} matches\n".format(len(matches))

# Calculate homography with RANSAC
pts_src = np.float32([kp1[a.queryIdx].pt for a in matches])
pts_dst = np.float32([kp2[b.trainIdx].pt for b in matches])

H, inliners = cv2.findHomography(pts_src, pts_dst, cv2.RANSAC, 1.0)
print H
print "\n {} inliners".format(len([a for a in inliners if a>0]))

# Translate
h1,w1 = img1.shape[:2]
h2,w2 = img2.shape[:2]
pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
pts2 = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
pts2_ = cv2.perspectiveTransform(pts2, H)
pts = np.concatenate((pts1, pts2_), axis=0)
[xmin, ymin] = np.int32(pts.min(axis=0).ravel())
[xmax, ymax] = np.int32(pts.max(axis=0).ravel())
t = [-xmin,-ymin]
Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) 
    
# Creating pano
img_dst = cv2.warpPerspective(img1_ori, Ht.dot(H), (xmax-xmin, ymax-ymin))
img_dst[t[1]:h1+t[1],t[0]:w1+t[0]] = img2_ori

# Didplay results
plt.figure(figsize=(15,20))
plt.subplot(121), plt.imshow(img1_ori), plt.title('img1')
plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(img2_ori), plt.title('img2')
plt.xticks([]), plt.yticks([])
plt.show()

plt.figure(figsize=(15,20))
plt.imshow(img_dst), plt.title('Pano stitches')
plt.xticks([]), plt.yticks([])
# plt.savefig('output_img/q1b_output.jpg')
plt.show()

# Question2

In [ ]:
img = cv2.imread('GB1.jpg', 0) 
img_ori = cv2.imread('GB1.jpg') 

def generate_random_centroid(img, num_of_cluster):
    cluster_centers = []
    for index in xrange(num_of_cluster):
        while True:
            random_center = random.randint(0,len(img)-1)
            if random_center not in cluster_centers:
                cluster_centers.append(random_center)
                break
    return np.array(cluster_centers)

def reassign_cluster_for_all_points(img, cluster_centers, clusters):
    total_reassign = 0
    new_clusters = [[] for _ in range(len(clusters))]
    for point_index in xrange(len(img)):
        new_clusters.append([])
        point_array = np.empty(len(cluster_centers))
        map(lambda x: img[point_index], point_array)
        cluster_center_points = np.array(map(lambda x: img[x], cluster_centers))
        cluster_index = np.argmin(abs(np.sqrt(point_array**2-cluster_center_points**2)))
        if point_index not in clusters[cluster_index]:
            total_reassign += 1
       
        new_clusters[cluster_index].append(point_index)
    return (np.array(new_clusters), total_reassign)

def recalculate_cluster_centers(img, clusters):
    cluster_centers = [[] for _ in range(len(clusters))]
    for i in xrange(len(clusters)):
        cluster_value = map(lambda x: img[x], clusters[i])
        avg_center = np.sum(cluster_value, axis=0)/len(cluster_value)
        avg_center_points = np.empty(len(clusters[i]))
        map(lambda x: avg_center, avg_center_points)
        print abs(np.sqrt(np.array(cluster_value)**2-np.array(avg_center_points)**2)).shape
        min_index = np.argmin(abs(np.sqrt(np.array(cluster_value)**2-np.array(avg_center_points)**2)))
        cluster_centers[i] = clusters[i][min_index]
    return cluster_centers

def perform_k_means(img, num_of_cluster):
    try:
        size = img.shape
        img.resize((size[0]*size[1], 3))
    except:
        img = img.flatten()
        
    cluster_centers = generate_random_centroid(img, num_of_cluster)
    clusters = [[] for _ in range(num_of_cluster)]
    
    while True:
        clusters, total_reassign = reassign_cluster_for_all_points(img, cluster_centers, clusters)
        if total_reassign == 0:
            break
        cluster_centers = recalculate_cluster_centers(img, clusters)
    return clusters
    
clusters = perform_k_means(img, 5)
print clusters
print 'done'

In [ ]:
img = cv2.imread('GB1.jpg', 0) 
img_ori = cv2.imread('GB1.jpg') 

def generate_random_centroid(img, num_of_cluster):
    cluster_centers = []
    for index in xrange(num_of_cluster):
        while True:
            random_center = random.randint(0,len(img)-1)
            if random_center not in cluster_centers:
                cluster_centers.append(random_center)
                break
    return np.array(cluster_centers)

def reassign_cluster_for_all_points(img, cluster_centers, clusters):
    total_reassign = 0
    new_clusters = [[] for _ in range(len(clusters))]
    for point_index in xrange(len(img)):
        new_clusters.append([])
        point_array = np.empty(len(cluster_centers))
        map(lambda x: img[point_index], point_array)
        cluster_center_points = np.array(map(lambda x: img[x], cluster_centers))
        cluster_index = np.argmin(abs(np.sqrt(point_array**2-cluster_center_points**2)))
        if point_index not in clusters[cluster_index]:
            total_reassign += 1
       
        new_clusters[cluster_index].append(point_index)
    return (np.array(new_clusters), total_reassign)

def recalculate_cluster_centers(img, clusters):
    cluster_centers = [[] for _ in range(len(clusters))]
    for i in xrange(len(clusters)):
        cluster_value = map(lambda x: img[x], clusters[i])
        avg_center = np.sum(cluster_value, axis=0)/len(cluster_value)
        avg_center_points = np.empty(len(clusters[i]))
        map(lambda x: avg_center, avg_center_points)
        print abs(np.sqrt(np.array(cluster_value)**2-np.array(avg_center_points)**2)).shape
        min_index = np.argmin(abs(np.sqrt(np.array(cluster_value)**2-np.array(avg_center_points)**2)))
        cluster_centers[i] = clusters[i][min_index]
    return cluster_centers

def perform_k_means(img, num_of_cluster):
    try:
        size = img.shape
        img.resize((size[0]*size[1], 3))
    except:
        img = img.flatten()
        
    cluster_centers = generate_random_centroid(img, num_of_cluster)
    clusters = [[] for _ in range(num_of_cluster)]
    
#     while True:
    clusters, total_reassign = reassign_cluster_for_all_points(img, cluster_centers, clusters)
#         if total_reassign == 0:
#             break
#         cluster_centers = recalculate_cluster_centers(img, clusters)
    print clusters
    
perform_k_means(img, 5)
